In [1]:
from roma_dspy import RecursiveSolverModule, RecursiveSolver, Executor, Atomizer, Planner, Aggregator
import dspy 

executor_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=True)
atomizer_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=True)
planner_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=True)
aggregator_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=True)

# Initialize modules
atomizer = Atomizer(lm=atomizer_lm)
planner = Planner(lm=planner_lm)
executor = Executor(lm=executor_lm)
aggregator = Aggregator(lm=aggregator_lm)

# Create solver
solver = RecursiveSolver(
    atomizer,
    planner,
    executor,
    aggregator,
    max_depth=1
)

dspy_module = RecursiveSolverModule(solver=solver)

In [2]:
task_goal = "Write me a blog post about the benefits of using DSPy."

root = await dspy_module.aforward(task_goal)

2025/09/30 19:27:55 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


In [3]:
from roma_dspy.visualizer import TreeVisualizer, ContextFlowVisualizer, LLMTraceVisualizer

# Show the full execution report
tree = TreeVisualizer(use_colors=True, show_timing=True)
print(tree.visualize(root.completed_task, dag=dspy_module._solver.last_dag))

📊 HIERARCHICAL TASK DECOMPOSITION TREE

└── ✅ [D0/1] Write me a blog post about the benefits of using DSPy. 📝PLAN [COMPLETED]
    ⏱️  6.68s
    🔍 atomizer: 1.1ms [$0.000302]
    📝 planner: 2.0ms [$0.001168]
    🔄 aggregator: 6.68s [10010/1960 tokens, $0.007903]
    💰 Node Total: 11970 tokens, $0.009373
    ├── ✅ [D1/1] Research DSPy to understand its core functionalities, use... ⚡EXECUTE [COMPLETED]
    │   ⏱️  0.5ms
    │   ⚡ executor: 0.5ms [$0.002332]
    ├── ✅ [D1/1] Outline the blog post structure, including an introductio... ⚡EXECUTE [COMPLETED]
    │   ⏱️  0.4ms
    │   ⚡ executor: 0.4ms [$0.004468]
    ├── ✅ [D1/1] Draft the introduction of the blog post, setting the stag... ⚡EXECUTE [COMPLETED]
    │   ⏱️  0.4ms
    │   ⚡ executor: 0.4ms [$0.001380]
    ├── ✅ [D1/1] Draft the body paragraphs, detailing each identified bene... ⚡EXECUTE [COMPLETED]
    │   ⏱️  0.4ms
    │   ⚡ executor: 0.4ms [$0.005076]
    ├── ✅ [D1/1] Draft the conclusion, summarizing the benefits and provid..

In [4]:
context_viz = ContextFlowVisualizer(use_colors=True)
print(context_viz.visualize(dspy_module, show_full_context=True))


🔗 CONTEXT FLOW VISUALIZATION (Index-Based)

────────────────────────────────────────────────────────────────────────────────
📦 Subgraph: Write me a blog post about the benefits of using DSPy....
────────────────────────────────────────────────────────────────────────────────

[Subtask 0] Research DSPy to understand its core functionalities, use cases, and p
  ℹ️  No dependencies (independent task)
  ✅ Result: DSPy is a programming model designed to simplify and optimize the development of applications using Large Language Models (LLMs). Its core functionali...
  ----------------------------------------------------------------------------

[Subtask 1] Outline the blog post structure, including an introduction, key benefi
  ⬅️  Dependencies: [0]
  📥 Context passed to LLM (XML format):
      <context>
      <subtask id="0">
        <goal>Research DSPy to understand its core functionalities, use cases, and primary benefits for LLM development.</goal>
        <output>DSPy is a programming m

In [8]:
viz = LLMTraceVisualizer(show_metrics=False, show_summary=False, verbose=True)
trace = viz.visualize(dspy_module)

In [9]:
print(trace)

=== EXECUTION TRACE ===
Root Goal: Write me a blog post about the benefits of using DSPy.
Max Depth: 1
Start Time: 2025-09-30T23:27:40.241707

[DEPTH 0] Task: Write me a blog post about the benefits of using DSPy.
  ID: 7d4886dd...

  MODULE: Atomizer
    Input:
      Write me a blog post about the benefits of using DSPy.
    Output:
      is_atomic: False
      node_type: "PLAN"

  MODULE: Planner
    Input:
      Write me a blog post about the benefits of using DSPy.
    Output:
      subtasks: [{goal: "Research DSPy to understand its core functionalities, use cases, and primary benefits for LLM development.", task_type: "RETRIEVE", dependencies: [], result: null, context_input: null}, {goal: "Outline the blog post structure, including an introduction, key benefit sections, and a conclusion. Identify specific benefits to highlight based on the research.", task_type: "THINK", dependencies: ["0"], result: null, context_input: null}, {goal: "Draft the introduction of the blog post, sett

In [7]:
# Debug: Check what modules were recorded for the root task
root_task = dspy_module._solver.last_dag.get_node(list(dspy_module._solver.last_dag.graph.nodes())[0])
print("Root task execution_history keys:", list(root_task.execution_history.keys()))
for module_name, result in root_task.execution_history.items():
    print(f"\n{module_name}:")
    print(f"  Duration: {result.duration:.2f}s")
    print(f"  Input type: {type(result.input)}")
    print(f"  Output type: {type(result.output)}")

Root task execution_history keys: ['atomizer', 'planner', 'aggregator']

atomizer:
  Duration: 0.00s
  Input type: <class 'str'>
  Output type: <class 'dict'>

planner:
  Duration: 0.00s
  Input type: <class 'str'>
  Output type: <class 'dict'>

aggregator:
  Duration: 6.68s
  Input type: <class 'dict'>
  Output type: <class 'str'>
